## Activitat DataFrames PySpark a partir dels  datasets:  flights,  planes i airports

In [140]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
flights_small_df = spark.read.csv("flights_small.csv", header=True)
flights_small_df = flights_small_df.withColumn("year", flights_small_df["year"].cast("integer"))
flights_small_df = flights_small_df.withColumn("month", flights_small_df["month"].cast("integer"))
flights_small_df = flights_small_df.withColumn("day", flights_small_df["day"].cast("integer"))
flights_small_df = flights_small_df.withColumn("dep_time", flights_small_df["dep_time"].cast("integer"))
flights_small_df = flights_small_df.withColumn("dep_delay", flights_small_df["dep_delay"].cast("integer"))
flights_small_df = flights_small_df.withColumn("arr_time", flights_small_df["arr_time"].cast("integer"))
flights_small_df = flights_small_df.withColumn("arr_delay", flights_small_df["arr_delay"].cast("integer"))
flights_small_df = flights_small_df.withColumn("flight", flights_small_df["flight"].cast("integer"))
flights_small_df = flights_small_df.withColumn("air_time", flights_small_df["air_time"].cast("integer"))
flights_small_df = flights_small_df.withColumn("distance", flights_small_df["distance"].cast("integer"))
flights_small_df = flights_small_df.withColumn("hour", flights_small_df["hour"].cast("integer"))
flights_small_df = flights_small_df.withColumn("minute", flights_small_df["minute"].cast("integer"))
flights_small_df = flights_small_df.withColumnRenamed("year", "flight_year")
flights_small_df.createOrReplaceTempView("flights_small_df")

### Estudi de la Petjada de Carboni en funció del temps a l’aire (Petjada de Carboni=airtime):

In [141]:
#1. Quin és l’avió («tailnum») amb major petjada de carboni.
query = 'SELECT tailnum, SUM(air_time) AS air_time FROM flights_small_df GROUP BY tailnum ORDER BY air_time DESC LIMIT 1'
spark.sql(query).show()

+-------+--------+
|tailnum|air_time|
+-------+--------+
| N435AS|    7834|
+-------+--------+



In [142]:
#2. Quin és l’avió («tailnum») amb menor petjada de carboni. ¿ha estat temps sense volar?
query = 'SELECT tailnum, SUM(air_time) AS air_time FROM flights_small_df WHERE air_time IS NOT NULL GROUP BY tailnum ORDER BY air_time ASC LIMIT 1'
spark.sql(query).show()

+-------+--------+
|tailnum|air_time|
+-------+--------+
| N658CT|      27|
+-------+--------+



In [143]:
#3. Quina és la petjada promig i la petjada mitjana segons «tailnum». Si hi aprecieu diferència notable a què ho atribuiu?
query = 'SELECT tailnum, ROUND(AVG(air_time), 2) AS air_time_avg, ROUND(MEAN(air_time), 2) AS air_time_mean FROM flights_small_df GROUP BY tailnum'
spark.sql(query).show()

+-------+------------+-------------+
|tailnum|air_time_avg|air_time_mean|
+-------+------------+-------------+
| N442AS|      177.45|       177.45|
| N102UW|       265.5|        265.5|
| N36472|      179.75|       179.75|
| N38451|      219.25|       219.25|
| N73283|      131.75|       131.75|
| N513UA|       199.0|        199.0|
| N954WN|        98.4|         98.4|
| N388DA|      208.33|       208.33|
| N567AA|       199.0|        199.0|
| N516UA|       191.0|        191.0|
| N927DN|       175.0|        175.0|
| N8322X|       224.0|        224.0|
| N466SW|       158.0|        158.0|
|  N6700|       249.0|        249.0|
| N607AS|      129.51|       129.51|
| N622SW|        97.0|         97.0|
| N584AS|      201.42|       201.42|
| N914WN|       125.5|        125.5|
| N654AW|       130.5|        130.5|
| N336NW|        80.0|         80.0|
+-------+------------+-------------+
only showing top 20 rows



In [144]:
#4. Quina és l’aerolinia amb major petjada de carboni.
query = 'SELECT carrier, SUM(air_time) AS air_time FROM flights_small_df GROUP BY carrier ORDER BY air_time DESC LIMIT 1'
spark.sql(query).show()

+-------+--------+
|carrier|air_time|
+-------+--------+
|     AS|  594898|
+-------+--------+



In [145]:
#5. Quina és l’aerolinia amb menor petjada de carboni.
query = 'SELECT carrier, SUM(air_time) AS air_time FROM flights_small_df GROUP BY carrier ORDER BY air_time ASC LIMIT 1'
spark.sql(query).show()

+-------+--------+
|carrier|air_time|
+-------+--------+
|     VX|   19990|
+-------+--------+



In [146]:
#6. Quin és el fabricant d’avions amb major petjada de carboni.
planes_df = spark.read.csv("planes.csv", header=True)
planes_df = planes_df.withColumn("year", planes_df["year"].cast("integer"))
planes_df = planes_df.withColumn("engines", planes_df["engines"].cast("integer"))
planes_df = planes_df.withColumn("seats", planes_df["seats"].cast("integer"))
planes_df = planes_df.withColumn("speed", planes_df["speed"].cast("integer"))
flights_planes_df = flights_small_df.join(planes_df, flights_small_df.tailnum == planes_df.tailnum, 'inner')
flights_planes_df.createOrReplaceTempView("flights_planes_df")
query = 'SELECT manufacturer, SUM(air_time) AS air_time FROM flights_planes_df GROUP BY manufacturer ORDER BY air_time DESC LIMIT 1'
spark.sql(query).show()

+------------+--------+
|manufacturer|air_time|
+------------+--------+
|      BOEING| 1044572|
+------------+--------+



In [147]:
#7. Quin és el fabricants d’avions amb menor petjada de carboni.
query = 'SELECT manufacturer, SUM(air_time) AS air_time FROM flights_planes_df GROUP BY manufacturer ORDER BY air_time ASC LIMIT 1'
spark.sql(query).show()

+-------------+--------+
| manufacturer|air_time|
+-------------+--------+
|BARKER JACK L|     131|
+-------------+--------+



In [148]:
#8. Quin és el model d’avions amb major petjada de carboni.
query = 'SELECT manufacturer, model, SUM(air_time) AS air_time FROM flights_planes_df GROUP BY manufacturer, model ORDER BY air_time DESC LIMIT 1'
spark.sql(query).show()

+------------+-------+--------+
|manufacturer|  model|air_time|
+------------+-------+--------+
|      BOEING|737-890|  260964|
+------------+-------+--------+



In [149]:
#9. Quin és el model d’avions amb menor petjada de carboni.
query = 'SELECT manufacturer, model, SUM(air_time) AS air_time FROM flights_planes_df GROUP BY manufacturer, model ORDER BY air_time ASC LIMIT 1'
spark.sql(query).show()

+------------+-------+--------+
|manufacturer|  model|air_time|
+------------+-------+--------+
|      BOEING|737-3T5|     104|
+------------+-------+--------+



In [150]:
#10. Fes una taula de petjada de carboni en funció dels anys de fabricació dels avions
query = 'SELECT year, SUM(air_time) AS air_time FROM flights_planes_df GROUP BY year ORDER BY year'
spark.sql(query).show()

+----+--------+
|year|air_time|
+----+--------+
|null|   15556|
|   0|     265|
|1959|     199|
|1975|     195|
|1980|     598|
|1984|    1229|
|1985|    4900|
|1986|    2220|
|1987|    5939|
|1988|    9510|
|1989|    7998|
|1990|   15555|
|1991|   11563|
|1992|   39326|
|1993|   11901|
|1994|   20904|
|1995|   14143|
|1996|   19833|
|1997|   33159|
|1998|   71523|
+----+--------+
only showing top 20 rows



In [151]:
#11. Quina és l’aerolinia amb major petjada de carboni normalitzat al número de seients disponible a cada vol.
query = 'SELECT carrier, SUM(air_time) / SUM(seats) AS air_time_per_seat FROM flights_planes_df GROUP BY carrier ORDER BY air_time_per_seat DESC LIMIT 1'
spark.sql(query).show()

+-------+------------------+
|carrier| air_time_per_seat|
+-------+------------------+
|     AA|1.3147003879867798|
+-------+------------------+



In [152]:
#12. Quina és l’aerolinia amb menor petjada de carboni normalitzat al número de seients disponible a cada vol.
query = 'SELECT carrier, SUM(air_time) / SUM(seats) AS air_time_per_seat FROM flights_planes_df GROUP BY carrier ORDER BY air_time_per_seat ASC LIMIT 1'
spark.sql(query).show()

+-------+------------------+
|carrier| air_time_per_seat|
+-------+------------------+
|     VX|0.6934230609130012|
+-------+------------------+



In [153]:
#13. Quin és el fabricant d’avions amb major petjada de carboni normalitzat al número de seients disponible a cada vol.
query = 'SELECT manufacturer, SUM(air_time) / SUM(seats) AS air_time_per_seat FROM flights_planes_df GROUP BY manufacturer ORDER BY air_time_per_seat DESC LIMIT 1'
spark.sql(query).show()

+-------------+-----------------+
| manufacturer|air_time_per_seat|
+-------------+-----------------+
|BARKER JACK L|             65.5|
+-------------+-----------------+



In [154]:
#14. Quin és el fabricants d’avions amb menor petjada de carboni normalitzat al número de seients disponible a cada vol.
query = 'SELECT manufacturer, SUM(air_time) / SUM(seats) AS air_time_per_seat FROM flights_planes_df GROUP BY manufacturer ORDER BY air_time_per_seat ASC LIMIT 1'
spark.sql(query).show()

+------------+------------------+
|manufacturer| air_time_per_seat|
+------------+------------------+
|      AIRBUS|0.8025864420749581|
+------------+------------------+



In [155]:
#15. Quin és el model d’avions amb major petjada de carboni normalitzat al número de seients disponible a cada vol.
query = 'SELECT manufacturer, model, SUM(air_time) / SUM(seats) AS air_time_per_seat FROM flights_planes_df GROUP BY manufacturer, model ORDER BY air_time_per_seat DESC LIMIT 1'
spark.sql(query).show()

+------------+-----+-----------------+
|manufacturer|model|air_time_per_seat|
+------------+-----+-----------------+
|      CESSNA|  150|             99.5|
+------------+-----+-----------------+



In [156]:
#16. Quin és el model d’avions amb menor petjada de carboni normalitzat al número de seients disponible a cada vol.
query = 'SELECT manufacturer, model, SUM(air_time) / SUM(seats) AS air_time_per_seat FROM flights_planes_df GROUP BY manufacturer, model ORDER BY air_time_per_seat ASC LIMIT 1'
spark.sql(query).show()

+------------+-------+-----------------+
|manufacturer|  model|air_time_per_seat|
+------------+-------+-----------------+
|      BOEING|737-3T5|0.348993288590604|
+------------+-------+-----------------+



In [157]:
#17. Fes una taula de petjada de carboni en funció dels anys de fabricació dels avions normalitzat al número de seients disponible a cada vol.
query = 'SELECT year, SUM(air_time) / SUM(seats) AS air_time_per_seat FROM flights_planes_df GROUP BY year ORDER BY year'
spark.sql(query).show()

+----+------------------+
|year| air_time_per_seat|
+----+------------------+
|null|1.0381047714381049|
|   0|        1.03515625|
|1959|              99.5|
|1975|              39.0|
|1980|24.916666666666668|
|1984|0.6904494382022472|
|1985|0.8541049328917553|
|1986|0.8771236665349664|
|1987|0.8649868919312554|
|1988|0.6520844761382337|
|1989|0.8479643765903307|
|1990|0.8448294590484466|
|1991| 0.816423074207442|
|1992|0.8945859872611465|
|1993|0.8149137222678718|
|1994|0.8245828566920437|
|1995|0.8277537164930352|
|1996|0.8999047143699805|
|1997|0.9564452392627419|
|1998|0.8082882231288212|
+----+------------------+
only showing top 20 rows



In [158]:
#19. Surt a compte des del punt de vista de la petjada de cada passatger que els avions tinguin més seients, menys o no és una hipòtesi vàlida?
query = 'SELECT seats, air_time FROM flights_planes_df'
spark.sql(query).show()
print("No es una hipòtesi vàlida, ja que hi ha avions amb molts seients i petjada de carboni baixa i avions amb pocs seients i petjada de carboni alta.")

+-----+--------+
|seats|air_time|
+-----+--------+
|  182|     132|
|  149|     360|
|  182|     111|
|  149|      83|
|  151|     127|
|  149|     121|
|  140|      90|
|  182|      98|
|  149|     135|
|  149|     198|
|  149|     130|
|  149|     154|
|  222|     127|
|  149|     183|
|  149|     129|
|  149|      90|
|  151|      76|
|  140|     216|
|  149|     290|
|   80|     111|
+-----+--------+
only showing top 20 rows

No es una hipòtesi vàlida, ja que hi ha avions amb molts seients i petjada de carboni baixa i avions amb pocs seients i petjada de carboni alta.


### Estudi de la Puntualitat

In [159]:
#1. Quina és l’aerolinia amb més retard?
query = 'SELECT carrier, SUM(arr_delay) AS arr_delay FROM flights_small_df GROUP BY carrier ORDER BY arr_delay DESC LIMIT 1'
spark.sql(query).show()

+-------+---------+
|carrier|arr_delay|
+-------+---------+
|     WN|     8373|
+-------+---------+



In [160]:
#2. Quina és l’aerolinia amb més retard normalitzat al número de vols operats?
query = 'SELECT carrier, SUM(arr_delay) / COUNT(arr_delay) AS arr_delay_per_flight FROM flights_small_df GROUP BY carrier ORDER BY arr_delay_per_flight DESC LIMIT 1'
spark.sql(query).show()

+-------+--------------------+
|carrier|arr_delay_per_flight|
+-------+--------------------+
|     AA|    6.51890756302521|
+-------+--------------------+



In [161]:

#3. Quina és l’aerolinia amb més retard normalitzat al número d’avions disponibles? Afecta tenir més o menys avions (flota)?

#4. Fes una taula: en una columna anys de posada en servei i en l’altra de retard. Es pot deduir que l’edat de l’avió afecta a la puntualitat?

#6. Agafem la taula anterior i hi afegim la granularitat de l'aerolínia és a dir estudiem l’edat dels avions per cada aerolínia. Es pot fer alguna hipótesi relacionada amb la gestió d’alguna aerolínia?

#8. Agafem la taula anterior i enlloc de granular per aerolínia granulem per «seats». Es pot deduir que a major nombre de seients disponibles més retard? Hi ha colinialitat?

#10. Fes un ó dos passos més enllà. És a dir, si això fos una entrevista de treball per una aerolínia, presenta una o dues hipótesis més sobre la puntualitat a partir de les dades d’aquests Datasets.
